In [13]:
import talib
# import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5

In [23]:
# MT5 connection

if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
login = 114781990
password = 'Mypassword$1234'
server = 'Exness-MT5Trial6'

mt5.login(login, password, server)

True

In [24]:
# Downloading data and creating two datasets

rate = mt5.copy_rates_from('EURUSDm', mt5.TIMEFRAME_M15, datetime.datetime.now(), 500)
nf = pd.DataFrame(rate)
        
         
nf['date']=pd.to_datetime(nf['time'], unit='s')

nf.set_index(np.arange(len(nf)), inplace = True)
nf = nf.drop(['spread', 'real_volume', 'tick_volume', 'time'], axis='columns')


df = nf.iloc[:50]  # First half
df2 = nf.iloc[50:]  # Second half



In [25]:
# Past trend of last 21 candles
# catching past trend index date

prev_high, prev_low, current_high, current_low = float(), float(), float(), float()

trend_date  = None
prev_trend = None

# Pass atleast 50 candles

def past_trend(df):
    
    global prev_high, prev_low, current_high, current_low, trend_date, prev_trend
    
    length = len(df)
    
    # Calculate max and min high values
    prev_high = df.iloc[length - 42 : length - 21]['high'].max()
    prev_low = df.iloc[length - 42 : length - 21]['low'].min()
    
    current_high = df.iloc[length - 21 : length]['high'].max()
    current_low = df.iloc[length - 21 : length]['low'].min()
    
#     Up condition
    
    if current_high > prev_high and current_low > prev_low:
        
        df.loc[length - 21 : length, 'trend'] = 'up'
        
#    Down condition     
    elif current_high < prev_high and current_low < prev_low:
        
        df.loc[length - 21 : length, 'trend'] = 'down'
        
    else:
        
        df.loc[length - 21 : length, 'trend'] = 'no trend'
        
        
    prev_high = current_high
    prev_low = current_low
    trend_date = df['date'].iloc[-1]
    prev_trend = df['trend'].iloc[-1]
        
        


In [26]:
# Current Trend
# Get trend of last 42 days and then compare it to 
# recent candle with 21 candles window

bull_no_trend = False
bear_no_trend = False

no_trend_up = False
no_trend_down = False

def test_current_trend(high, low, l):
    
    global prev_high, prev_low, prev_trend, bull_no_trend, bear_no_trend, no_trend_up, no_trend_down
    
#     When Prev trend is up and Prev low is defended

    if prev_trend == 'up' and bull_no_trend == False:
    
        if low > prev_low:

            df.loc[l, 'trend'] = 'up' 
        
        else:
            
            df.loc[l, 'trend'] = 'no trend bull' 
            bull_no_trend = True
            
#   When Prev tend is up but prev low is broken

    elif prev_trend == 'up' and bull_no_trend == True:
        
        df.loc[l, 'trend'] = 'no trend bull 1' 

#   When Prev trend is down and high is defended
    elif prev_trend == 'down' and bear_no_trend == False:
        
        if high < prev_high:

            df.loc[l, 'trend'] = 'down'
        
        else:
            
            df.loc[l, 'trend'] = 'no trend bear' 
            bear_no_trend = True

        
        
#   When Prev trend is down but high is broken

    elif prev_trend == 'down' and bear_no_trend == True:
        
        df.loc[l, 'trend'] = 'no trend bear1'

#     When Prev trend no trend

    elif prev_trend == 'no trend' and no_trend_up == False and no_trend_down == False:

#    Need to keep remaining 21 batch up 
#    until low break prev low
        
        if high > prev_high:
            
            df.loc[l, 'trend'] = 'up'
            no_trend_up = True
            
#    Need to keep remaining 21 batch down
#    until high break prev high
        elif low < prev_low:
            
            df.loc[l, 'trend'] = 'down'
            no_trend_down = True
            
        else:
            
            df.loc[l, 'trend'] = 'no trend'
            
#   Till prev low is not broken keep trend up
    
    elif prev_trend == 'no trend' and no_trend_up == True:
        
        if low > prev_low:
            
            df.loc[l, 'trend'] = 'up'
            
        else:
            
            df.loc[l, 'trend'] = 'no trend'
            no_trend_up = False
            
#   Till prev high is not broken keep the trend down

    elif prev_trend == 'no trend' and no_trend_down == True:
        
        if high < prev_high:
            
            df.loc[l, 'trend'] = 'down'
            
        else:
            
            df.loc[l, 'trend'] = 'no trend'
            no_trend_down = False
            
            


        
# In Position_index need pass index whose date is trend_date
def calculate_current_trend( position_index, days=21):

    global prev_high, prev_low, df, trend_date
    global prev_trend, bull_no_trend, bear_no_trend, no_trend_up, no_trend_down
    
    l = len(df) - 1

    if position_index + days > l:
        
        test_current_trend(df.iloc[l]['high'], df.iloc[l]['low'], l)
    
#      When 21 candles set is complete
    elif position_index + days == l:
        
        current_high = df.loc[df.index[position_index: l + 1], 'high'].max()
        current_low = df.loc[df.index[position_index: l + 1], 'low'].min()
        
        if current_high > prev_high and current_low > prev_low:
            df.loc[l, 'trend'] = 'up'
        
        elif current_high < prev_high and current_low < prev_low:
            df.loc[l, 'trend'] = 'down'
        
        else:
            df.loc[l, 'trend'] = 'no trend'
        
        trend_date = df['date'].iloc[-1]
        prev_high = current_high
        prev_low = current_low
        prev_trend = df['trend'].iloc[-1]

#         Resetting all the Flags
        bull_no_trend, bear_no_trend, no_trend_up, no_trend_down = [False] * 4

    


In [27]:
# df = 50 and df2 = 450
# Need to pass df for past_trend

past_trend(df)

# df.to_excel('test_trend_past.xlsx')


C:\Users\hp\AppData\Local\Temp\ipykernel_4052\3463352202.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[length - 21 : length, 'trend'] = 'no trend'


In [28]:
#  Need to find position_index
# Need to provide position_index from trend_date

past_df = pd.DataFrame()

for index, row in df2.iterrows():
    
#   Add live data

    df = df.append(row, ignore_index=True)
    
#   delete first row if greater than 200

    if len(df) > 200:
#         Saving the dropped row
        past_df = past_df.append(df.iloc[0], ignore_index=True)
        
        df = df.iloc[1:].reset_index(drop=True)
        
        
#     Find trend_date index and pass in calculate_current_trend
    
    position_index = df[df['date'] == trend_date].index[0]
    
    calculate_current_trend(position_index)
    
    
result_df = pd.concat([past_df, df], ignore_index=True)

result_df.to_excel('result.xlsx')

C:\Users\hp\AppData\Local\Temp\ipykernel_4052\938138352.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_4052\938138352.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  past_df = past_df.append(df.iloc[0], ignore_index=True)
